In [1]:
#written by Zilin Shen and Daniel de Mello
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import time
import numpy as np
from packet_capturer_daniel import PacketCapturer
from window_manager import WindowManager
from feature_extractor import FeatureExtractor
from numpy_generator import NumpyGenerator
from lstm import LSTM
from label_packet import generate_label_data, label_packets
from random_perturb import random_perturb
from pso import PSO
from lstm import Lstm


# the hyperparameters of window manager
window_size = 1
move_size = 0.1 #how long the movement of window
swnd = True
# the class of detector
# attack, infection,reconnaissance
kind = "reconnaissance"
# the chosen algorithm "feedforward" or "lstm"
algorithm = "lstm"

# -----------get the preprocessed training and testing saved as .npy files
test_label_infection = np.load('preprocessed/test_label_infection.npy')
train_label_infection = np.load('preprocessed/train_label_infection.npy')
test_data_infection = np.load('preprocessed/test_data_infection.npy')
train_data_infection = np.load('preprocessed/train_data_infection.npy')

test_label_reconnaissance = np.load('preprocessed/test_label_reconnaissance.npy')
train_label_reconnaissance = np.load('preprocessed/train_label_reconnaissance.npy')
test_data_reconnaissance = np.load('preprocessed/test_data_reconnaissance.npy')
train_data_reconnaissance = np.load('preprocessed/train_data_reconnaissance.npy')

test_label_attack = np.load('preprocessed/test_label_attack.npy')
train_label_attack = np.load('preprocessed/train_label_attack.npy')
test_data_attack = np.load('preprocessed/test_data_attack.npy')
train_data_attack = np.load('preprocessed/train_data_attack.npy')

all_data = {"infection": 
                        {
                        'train': [train_data_infection, train_label_infection], 
                        'test': [test_data_infection, test_label_infection]
                        },
            "attack": 
                        {
                        'train': [train_data_attack, train_label_attack], 
                        'test': [test_data_attack, test_label_attack]
                        },
            "reconnaissance": 
                        {
                        'train': [train_data_reconnaissance, train_label_reconnaissance], 
                        'test': [test_data_reconnaissance, test_label_reconnaissance]
                        }
            }

test_label_infection_unb = np.load('preprocessed/test_label_infection_unbalanced.npy')
train_label_infection_unb = np.load('preprocessed/train_label_infection_unbalanced.npy')
test_data_infection_unb = np.load('preprocessed/test_data_infection_unbalanced.npy')
train_data_infection_unb = np.load('preprocessed/train_data_infection_unbalanced.npy')

infection_unb_data = {'train': [train_data_infection_unb, train_label_infection_unb],
                      'test': [test_data_infection_unb, test_label_infection_unb],
                     }
# -----------------start learning with some algorithm----------------------

def print_header(string):
    bar_len = len(string)
    bar = '-'*bar_len
    print(bar+ '\n' + string + '\n' + bar)

def train_detector(detector, data, kind):

    #train data
    train_data = data['train']
    train_examples = train_data[0]
    train_labels = train_data[1]

    #test data
    test_data = data['test']
    test_examples = test_data[0]
    test_labels = test_data[1]
        
    features_len = train_examples.shape[1]
    print('features len is ', features_len)
    
    print_header("Training {} detector".format(kind))
    detector.learning(features_len, train_examples, train_labels)
                    
    print_header("Measureing {} detector performance on test data".format(kind))
    detector.detection(test_examples, test_labels)
    return detector 



2023-04-27 05:02:02.776920: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-27 05:02:02.776945: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


the count of label lines 20084
(20084,)


In [2]:
all_data['infection']['train'][1].sum()/len(all_data['infection']['train'][1])

0.5

In [3]:
infection_unb_data['train'][0].shape

(10016, 41)

In [4]:
attack_detector = train_detector(Lstm("lstm-attack"), all_data['attack'], 'attack')
reconnaissance_detector = train_detector(Lstm("lstm-recon"), all_data['reconnaissance'], 'reconnaissance')
infection_detector = train_detector(Lstm("lstm-infection"), all_data['infection'], 'infection')

features len is  41
------------------------
Training attack detector
------------------------


2023-04-27 05:02:09.357327: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-04-27 05:02:09.358148: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-04-27 05:02:09.422040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 715.34GiB/s
2023-04-27 05:02:09.422139: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/oracle/instantclient_19_8:/opt/oracle/instantclient_19_8:/p/cuda-11.5
2023-04-27 05:02:09.422185: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 

the info of dataset, (19152, 1, 41)
shuffle is enabled for training
Epoch 1/50
539/539 - 4s - loss: 0.1382 - accuracy: 0.9513 - auc: 0.9888 - recall: 0.9583 - precision: 0.9450 - val_loss: 0.0674 - val_accuracy: 0.9760 - val_auc: 0.9963 - val_recall: 0.9897 - val_precision: 0.9640
Epoch 2/50
539/539 - 1s - loss: 0.0654 - accuracy: 0.9760 - auc: 0.9966 - recall: 0.9872 - precision: 0.9654 - val_loss: 0.0518 - val_accuracy: 0.9828 - val_auc: 0.9984 - val_recall: 0.9928 - val_precision: 0.9738
Epoch 3/50
539/539 - 1s - loss: 0.0572 - accuracy: 0.9791 - auc: 0.9972 - recall: 0.9893 - precision: 0.9695 - val_loss: 0.0617 - val_accuracy: 0.9791 - val_auc: 0.9975 - val_recall: 0.9979 - val_precision: 0.9624
Epoch 4/50
539/539 - 1s - loss: 0.0546 - accuracy: 0.9805 - auc: 0.9973 - recall: 0.9916 - precision: 0.9700 - val_loss: 0.0472 - val_accuracy: 0.9823 - val_auc: 0.9985 - val_recall: 0.9959 - val_precision: 0.9700
Epoch 5/50
539/539 - 1s - loss: 0.0520 - accuracy: 0.9811 - auc: 0.9975 - re

In [5]:
train_events_per_detector = []
for detector in  [attack_detector, infection_detector, reconnaissance_detector]:
    #examples_train = infection_unb_data['train'][0]
    examples_train = all_data['infection']['train'][0]
    train_events = detector.predict(examples_train)
    print('proportion of events tagged by {} detector: '.format(detector.get_name()), np.sum(np.array(train_events)>0.5)/len(train_events))
    train_events_per_detector.append(train_events)

benign_events = []
no_events = len(train_events_per_detector[0])
for i  in range(no_events):
    not_attack = 1-train_events_per_detector[0][i]
    not_infection = 1-train_events_per_detector[1][i]
    not_recon = 1-train_events_per_detector[2][i]
    benign_events.append(not_attack * not_infection * not_recon)
train_events_per_detector.append(benign_events)
print('proportion of events estimated to be benign : ', np.sum(np.array(benign_events)>0.5)/len(benign_events))
train_events_per_detector = np.array(train_events_per_detector).squeeze()


proportion of events tagged by lstm-attack detector:  0.041174689676052076
proportion of events tagged by lstm-infection detector:  0.424260773034615
proportion of events tagged by lstm-recon detector:  0.4770915329498436
proportion of events estimated to be benign :  0.0686749419719447


In [6]:
def softmax(x):
    """
    Compute the softmax of vector x.
    """
    # Subtract the maximum value of x from each element of x to avoid overflow
    # and make the softmax numerically stable.
    z = x - np.max(x)
    
    # Calculate the exponential of each element in z.
    exp_z = np.exp(z)
    
    # Calculate the sum of the exponential values.
    sum_exp_z = np.sum(exp_z)
    
    # Divide each element of the exponential vector by the sum of exponential values.
    softmax_z = exp_z / sum_exp_z
    
    return softmax_z

train_events_softmax = []
for i in range(no_events):
    softmax_event = softmax(train_events_per_detector[:,i])
    train_events_softmax.append(softmax_event)

In [7]:
np.sum(np.argmax(train_events_softmax, axis=-1)==0)/no_events

0.031738823291956805

In [8]:
train_events_softmax

[array([0.17114225, 0.46518672, 0.19252035, 0.17115071], dtype=float32),
 array([0.1748777 , 0.1748777 , 0.1748777 , 0.47536692], dtype=float32),
 array([0.36709303, 0.36164817, 0.13562383, 0.13563497], dtype=float32),
 array([0.17537059, 0.47203344, 0.17549047, 0.17710559], dtype=float32),
 array([0.17479001, 0.4433771 , 0.19600382, 0.18582906], dtype=float32),
 array([0.18235676, 0.4436341 , 0.17870024, 0.19530885], dtype=float32),
 array([0.1748777 , 0.1748777 , 0.47536692, 0.1748777 ], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823029], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823029], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823029], dtype=float32),
 array([0.17488089, 0.17488089, 0.47534677, 0.17489153], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823029], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823029], dtype=float32),
 array([0.17746069, 0.22156726, 0.39274186, 0.20823

In [9]:
from perm_seq2seq_attention import PermSeq2seqAttention

In [10]:
seq2seq = PermSeq2seqAttention('seq2seq')

In [11]:
train_events_softmax[4]

array([0.17479001, 0.4433771 , 0.19600382, 0.18582906], dtype=float32)

In [12]:
config = {"sequence_length": 10, 
          "permute_truncated": False,
          "use_prob_embedding": False,
          "rv": 1
        }

In [13]:
train_label_infection_unb.sum()

107.0

In [14]:
#seq2seq.learning(train_events_softmax, train_label_infection_unb, config)
seq2seq.learning(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is False
Use prob embedding is False
Permute Truncated is disabled
truncated idxs are [[ 0  1  2  3  4  5  6  7  8  9]
 [ 1  2  3  4  5  6  7  8  9 10]
 [ 2  3  4  5  6  7  8  9 10 11]
 [ 3  4  5  6  7  8  9 10 11 12]
 [ 4  5  6  7  8  9 10 11 12 13]
 [ 5  6  7  8  9 10 11 12 13 14]
 [ 6  7  8  9 10 11 12 13 14 15]
 [ 7  8  9 10 11 12 13 14 15 16]
 [ 8  9 10 11 12 13 14 15 16 17]
 [ 9 10 11 12 13 14 15 16 17 18]
 [10 11 12 13 14 15 16 17 18 19]
 [11 12 13 14 15 16 17 18 19 20]
 [12 13 14 15 16 17 18 19 20 21]
 [13 14 15 16 17 18 19 20 21 22]
 [14 15 16 17 18 19 20 21 22 23]
 [15 16 17 18 19 20 21 22 23 24]
 [16 17 18 19 20 21 22 23 24 25]
 [17 18 19 20 21 22 23 24 25 26]
 [18 19 20 21 22 23 24 25 26 27]
 [19 20 21 22 23 24 25 26 27 28]]
x_in is [[[0.17114225 0.46518672 0.19252035 0.17115071]
  [0.1748777  0.1748777  0.1748777  0.47536692]
  [0.36709303 0.36164817 0.13562383 0.13563497]
  [0.17537059 0.47203344 0.17549047 0.17710559]
  [0.17479001 0.4433771  0.19600382

In [15]:
res_probs, res_idxs = seq2seq.analysis(train_events_softmax, all_data['infection']['train'][1], config)

Permute truncated is False
Use prob embedding is False


In [16]:
train_events_per_detector = []
for detector in  [attack_detector, infection_detector, reconnaissance_detector]:
    #examples_train = infection_unb_data['train'][0]
    examples_train = all_data['infection']['train'][0]
    train_events = detector.predict(examples_train)
    print('proportion of events tagged by {} detector: '.format(detector.get_name()), np.sum(np.array(train_events)>0.5)/len(train_events))
    train_events_per_detector.append(train_events)

benign_events = []
no_events = len(train_events_per_detector[0])
for i  in range(no_events):
    not_attack = 1-train_events_per_detector[0][i]
    not_infection = 1-train_events_per_detector[1][i]
    not_recon = 1-train_events_per_detector[2][i]
    benign_events.append(not_attack * not_infection * not_recon)
train_events_per_detector.append(benign_events)
print('proportion of events estimated to be benign : ', np.sum(np.array(benign_events)>0.5)/len(benign_events))
train_events_per_detector = np.array(train_events_per_detector).squeeze()

proportion of events tagged by lstm-attack detector:  0.041174689676052076
proportion of events tagged by lstm-infection detector:  0.424260773034615
proportion of events tagged by lstm-recon detector:  0.4770915329498436
proportion of events estimated to be benign :  0.0686749419719447


In [17]:
iPS_pred = infection_detector.predict(all_data['infection']['train'][0])


In [18]:
infection_tagged_PS = []

for idx, pred in enumerate(iPS_pred):
    if pred>0.5:
        infection_tagged_PS.append(idx)


In [19]:
retrain_pos = []
retrain_neg = []
for idx in infection_tagged_PS:
    if idx in res_idxs:
        retrain_pos.append(idx)
    else:
        retrain_neg.append(idx)

In [20]:
retrain_labels = [0]*len(retrain_neg) + [1]*len(retrain_pos)
retrain_labels = np.array(retrain_labels)
retrain_data = all_data['infection']['train'][0][retrain_neg].tolist() + all_data['infection']['train'][0][retrain_pos].tolist()
retrain_data = np.array(retrain_data)

In [21]:
import copy
retrain_labels = copy.deepcopy(all_data['infection']['train'][1])
retrain_data = copy.deepcopy(all_data['infection']['train'][0])

for idx, l in enumerate(retrain_labels):
    if idx in retrain_pos:
        retrain_labels[idx] = 1
    if idx in retrain_neg:
        retrain_labels[idx] = 0


In [22]:
len(retrain_pos)

8374

In [23]:
np.sum(all_data['infection']['train'][1])

9909.0

In [24]:
np.sum(retrain_labels)

10012.0

In [25]:
def retrain_detector(detector, retrain_data, retrain_labels, test_data, test_labels):

        
    features_len = retrain_data.shape[1]
    print('features len is ', features_len)
    
    print_header("Retraining {} detector".format('infection'))
    detector.learning(features_len, retrain_data, retrain_labels)
                    
    print_header("Measureing {} detector performance on test data".format('infection'))
    detector.detection(test_data, test_labels)
    return detector 

In [26]:
len(retrain_labels)

19818

In [27]:
retrain_labels.shape

(19818,)

In [28]:
test_data =  all_data['infection']['test'][0]
test_labels =  all_data['infection']['test'][1]
train_data = all_data['infection']['train'][0]

infection_detector = retrain_detector(infection_detector, train_data, retrain_labels, test_data, test_labels)

features len is  41
-----------------------------
Retraining infection detector
-----------------------------
the info of dataset, (19818, 1, 41)
shuffle is enabled for training
Epoch 1/50
558/558 - 3s - loss: 0.3627 - accuracy: 0.8608 - auc_4: 0.9039 - recall_4: 0.8162 - precision_4: 0.8993 - val_loss: 0.2938 - val_accuracy: 0.8910 - val_auc_4: 0.9283 - val_recall_4: 0.8197 - val_precision_4: 0.9565
Epoch 2/50
558/558 - 1s - loss: 0.2707 - accuracy: 0.8961 - auc_4: 0.9343 - recall_4: 0.8276 - precision_4: 0.9616 - val_loss: 0.2717 - val_accuracy: 0.8951 - val_auc_4: 0.9345 - val_recall_4: 0.8308 - val_precision_4: 0.9538
Epoch 3/50
558/558 - 1s - loss: 0.2577 - accuracy: 0.8992 - auc_4: 0.9379 - recall_4: 0.8286 - precision_4: 0.9675 - val_loss: 0.2627 - val_accuracy: 0.8981 - val_auc_4: 0.9361 - val_recall_4: 0.8248 - val_precision_4: 0.9669
Epoch 4/50
558/558 - 1s - loss: 0.2513 - accuracy: 0.9018 - auc_4: 0.9403 - recall_4: 0.8302 - precision_4: 0.9713 - val_loss: 0.2563 - val_accu

In [29]:
infection_detector_new = Lstm("lstm-infection")

Measureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9522797070635484 precision: 0.7843137254901961 recall: 0.5031446540880503 f1: 0.6130268199233716 AUC: 0.7455187675205024
fp: 44 ,tp: 160 ,fn: 158 ,tn: 3871


easureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9522797070635484 precision: 0.7543859649122807 recall: 0.5408805031446541 f1: 0.63003663003663 AUC: 0.6239359181345734
fp: 56 ,tp: 172 ,fn: 146 ,tn: 3859

------------------------------------------------------
Measureing infection detector performance on test data
------------------------------------------------------
accuracy: 0.9520434679896055 precision: 0.757847533632287 recall: 0.5314465408805031 f1: 0.6247689463955638 AUC: 0.6181040507000168
fp: 54 ,tp: 169 ,fn: 149 ,tn: 3861


In [30]:
infection_detector_new = Lstm("lstm-infection")
infection_detector = retrain_detector(infection_detector, train_data, retrain_labels, test_data, test_labels)

features len is  41
-----------------------------
Retraining infection detector
-----------------------------
the info of dataset, (19818, 1, 41)
shuffle is enabled for training
Epoch 1/50
558/558 - 4s - loss: 0.3568 - accuracy: 0.8625 - auc_5: 0.9064 - recall_5: 0.8104 - precision_5: 0.9104 - val_loss: 0.2843 - val_accuracy: 0.8900 - val_auc_5: 0.9305 - val_recall_5: 0.8169 - val_precision_5: 0.9520
Epoch 2/50
558/558 - 1s - loss: 0.2685 - accuracy: 0.8967 - auc_5: 0.9346 - recall_5: 0.8305 - precision_5: 0.9620 - val_loss: 0.2675 - val_accuracy: 0.8996 - val_auc_5: 0.9347 - val_recall_5: 0.8241 - val_precision_5: 0.9662
Epoch 3/50
558/558 - 1s - loss: 0.2542 - accuracy: 0.9012 - auc_5: 0.9371 - recall_5: 0.8309 - precision_5: 0.9712 - val_loss: 0.2479 - val_accuracy: 0.9026 - val_auc_5: 0.9401 - val_recall_5: 0.8210 - val_precision_5: 0.9767
Epoch 4/50
558/558 - 1s - loss: 0.2477 - accuracy: 0.9024 - auc_5: 0.9407 - recall_5: 0.8301 - precision_5: 0.9750 - val_loss: 0.2547 - val_accu

In [5]:
len(all_data['reconnaissance']['train'][1])

10016

In [7]:
np.sum(all_data['reconnaissance']['train'][1]==0)

2811

In [8]:
np.sum(all_data['reconnaissance']['train'][1]==1)/len(all_data['reconnaissance']['train'][0])

0.7193490415335463

In [9]:
np.sum(all_data['attack']['train'][1]==1)/len(all_data['attack']['train'][0])

0.0439297124600639

In [10]:
np.sum(all_data['infection']['train'][1]==1)/len(all_data['infection']['train'][0])

0.010682907348242811